In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn
import seaborn as sns
from sklearn.svm import SVR
from sklearn.metrics import r2_score

In [2]:
df = pd.read_csv("autosmpgdata.csv")
df.head(10)

,MPG,CYLINDERS,DISPLACEMENT,HP,WEIGHT,ACCELERATION,YEAR,ORIGIN,NAME
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
5,15.0,8,429.0,198,4341,10.0,70,1,ford galaxie 500
6,14.0,8,454.0,220,4354,9.0,70,1,chevrolet impala
7,14.0,8,440.0,215,4312,8.5,70,1,plymouth fury iii
8,14.0,8,455.0,225,4425,10.0,70,1,pontiac catalina
9,15.0,8,390.0,190,3850,8.5,70,1,amc ambassador dpl


In [3]:
# La colonne HP n'est pas numérique
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MPG           398 non-null    float64
 1   CYLINDERS     398 non-null    int64  
 2   DISPLACEMENT  398 non-null    float64
 3   HP            398 non-null    object 
 4   WEIGHT        398 non-null    int64  
 5   ACCELERATION  398 non-null    float64
 6   YEAR          398 non-null    int64  
 7   ORIGIN        398 non-null    int64  
 8   NAME          398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


In [4]:
# Rendre la colonne HP numérique
df['HP'] = pd.to_numeric(df['HP'], errors = 'coerce') # ‘coerce’, NaN pour un  champs non valide

In [5]:
# La colonne HP contient 6 entrées non valides
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MPG           398 non-null    float64
 1   CYLINDERS     398 non-null    int64  
 2   DISPLACEMENT  398 non-null    float64
 3   HP            392 non-null    float64
 4   WEIGHT        398 non-null    int64  
 5   ACCELERATION  398 non-null    float64
 6   YEAR          398 non-null    int64  
 7   ORIGIN        398 non-null    int64  
 8   NAME          398 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 28.1+ KB


In [6]:
# Afficher les 6 lignes a supprimer
df[df['HP'].isna()]

,MPG,CYLINDERS,DISPLACEMENT,HP,WEIGHT,ACCELERATION,YEAR,ORIGIN,NAME
32,25.0,4,98.0,NaN,2046,19.0,71,1,ford pinto
126,21.0,6,200.0,NaN,2875,17.0,74,1,ford maverick
330,40.9,4,85.0,NaN,1835,17.3,80,2,renault lecar deluxe
336,23.6,4,140.0,NaN,2905,14.3,80,1,ford mustang cobra
354,34.5,4,100.0,NaN,2320,15.8,81,2,renault 18i
374,23.0,4,151.0,NaN,3035,20.5,82,1,amc concord dl


In [7]:
# Supprimer toutes les lignes non valides
df = df.dropna(axis = 0) # Si la valeur d'une cellule est NaN supprimer toute la ligne
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   MPG           392 non-null    float64
 1   CYLINDERS     392 non-null    int64  
 2   DISPLACEMENT  392 non-null    float64
 3   HP            392 non-null    float64
 4   WEIGHT        392 non-null    int64  
 5   ACCELERATION  392 non-null    float64
 6   YEAR          392 non-null    int64  
 7   ORIGIN        392 non-null    int64  
 8   NAME          392 non-null    object 
dtypes: float64(4), int64(4), object(1)
memory usage: 30.6+ KB


In [8]:
# La colonne 'NAME' n'est pas descriminante
# 392/301 ~ 1.30, trop de valeurs uniques, donc peu de valeurs qui se répètent
df['NAME'].value_counts()

amc matador            5
ford pinto             5
toyota corolla         5
toyota corona          4
amc hornet             4
                      ..
buick skyhawk          1
chevrolet monza 2+2    1
ford mustang ii        1
pontiac astro          1
chevy s-10             1
Name: NAME, Length: 301, dtype: int64

In [9]:
#stats = df.describe(percentiles = [0.1, 0.5,0.8])
stats = df.describe()
stats

,MPG,CYLINDERS,DISPLACEMENT,HP,WEIGHT,ACCELERATION,YEAR,ORIGIN
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,23.445918,5.471939,194.411990,104.469388,2977.584184,15.541327,75.979592,1.576531
std,7.805007,1.705783,104.644004,38.491160,849.402560,2.758864,3.683737,0.805518
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.000000,4.000000,105.000000,75.000000,2225.250000,13.775000,73.000000,1.000000
50%,22.750000,4.000000,151.000000,93.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,275.750000,126.000000,3614.750000,17.025000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


In [10]:
df = df.drop('NAME', axis = 1) # Colonne supprimée % des 392 enregistrements.

In [11]:
df.corr()

,MPG,CYLINDERS,DISPLACEMENT,HP,WEIGHT,ACCELERATION,YEAR,ORIGIN
MPG,1.000000,-0.777618,-0.805127,-0.778427,-0.832244,0.423329,0.580541,0.565209
CYLINDERS,-0.777618,1.000000,0.950823,0.842983,0.897527,-0.504683,-0.345647,-0.568932
DISPLACEMENT,-0.805127,0.950823,1.000000,0.897257,0.932994,-0.543800,-0.369855,-0.614535
HP,-0.778427,0.842983,0.897257,1.000000,0.864538,-0.689196,-0.416361,-0.455171
WEIGHT,-0.832244,0.897527,0.932994,0.864538,1.000000,-0.416839,-0.309120,-0.585005
ACCELERATION,0.423329,-0.504683,-0.543800,-0.689196,-0.416839,1.000000,0.290316,0.212746
YEAR,0.580541,-0.345647,-0.369855,-0.416361,-0.309120,0.290316,1.000000,0.181528
ORIGIN,0.565209,-0.568932,-0.614535,-0.455171,-0.585005,0.212746,0.181528,1.000000


In [ ]:
# Voyons la corrélation des autres variables et la colonne MPG pour sélectionner les meilleures attributs
sns.heatmap(df.corr(), annot = True, cmap = 'coolwarm')

<AxesSubplot:>

In [ ]:
# normalisation (valeur-moyenne)/std
df_normalized = ((df-df.mean())/df.std())
df_normalized

In [ ]:
#stats = df.describe(percentiles = [0.1, 0.5,0.8])
stats = df_normalized.describe()
stats

In [ ]:
data= df.drop('MPG', axis = 1)
target=df['MPG'].values

In [ ]:
# 85% données d'entrainement, 15% données de test
x_train,x_test,y_train,y_test=train_test_split(data, target, test_size=0.15,random_state=0)

In [ ]:
# Non-linear regression model using RBF SVM kernel, C=1, default value of gamma.
model = SVR (kernel='rbf',C=10000000)
model.fit(x_train.values, y_train)

In [ ]:
pred=model.predict(x_test.values)

In [ ]:
a=pd.DataFrame({'Real':y_test.reshape(-1),'Predict':pred.reshape(-1)})
a.head(25)

In [ ]:
print("training dataset: ", model.score(x_train.values, y_train))

In [ ]:
print("testing ", model.score(x_test.values, y_test))

In [ ]:
print("accuracy: ", r2_score(y_test,pred))

In [ ]:
from sklearn.metrics import mean_absolute_error
# Make predictions on the test set and calculate the mean absolute error
pred = model.predict(x_test.values)
mae = mean_absolute_error(y_test, pred)
print("Mean absolute error:", mae)

In [ ]:
def predict_mpg():
    car=[]
    cylinders = int(input("How many cylinders in your car: "))
    displacement = int(input("What is the displacement of your car: "))
    horsepower = int(input("What is the horsepower of your car: "))
    weight = int(input("What is the weight of your car: "))
    acceleration = int(input("What is the acceleration of your car: "))
    year = int(input("What is the year of your car: "))
    origin = int(input("What is the origin of your car: "))
    
    car.append(cylinders)
    car.append(displacement)
    car.append(horsepower)
    car.append(weight)
    car.append(acceleration)
    car.append(year)
    car.append(origin)
    
    x_text = [car]
    mpg = model.predict(x_text)
    print("We predict that your car will consum {} MPG".format(mpg[0]))

In [ ]:
predict_mpg()